#Classificação automatizada de tomates

Este projeto tem como base a criaçãod e uma classificação automatica de tomates verdes e maduros com a utilização da linguagem de programação python e suas bibliotecas.


##1º passo - Criar um script para buscar imagens na internet

Nesta etapa fiz uma busca através do script de forma separada por fotos de tomates verdes e tomates maduros Para tal pesquisa foi utilizado a biblioteca python bing_imagem_downloader


In [4]:
from bing_image_downloader import downloader

search = 'tomate maduro'
search2 = 'tomate verde'

downloader.download(search, limit=500,  output_dir='dataset', adult_filter_off=True, force_replace=False, verbose=True)
downloader.download(search2, limit=500,  output_dir='dataset', adult_filter_off=True, force_replace=False, verbose=True)

[%] Downloading Images to /home/antoniosvj/Projetos_Python/automatizacao_classificacao_tomate/dataset/tomate maduro


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://s3.dia.es/medias/h35/h3e/8834226061342.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://3.bp.blogspot.com/-zrEYCmthErI/TpNP29en6hI/AAAAAAAAHsQ/UWKUt4e-9n4/s1600/9.jpg


[%] File Downloaded !

[%] Downloading Image #3 from https://pimpamfruit.com/210-thickbox_default/tomate-maduro.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://biozaki.org/wp-content/uploads/tomate-maduro-biozaki.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://blog.mfrural.com.br/wp-content/uploads/2020/05/tomate-1024x654.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://fruitagash.es/372-large_default/tomate-maduro-super-oferta.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://i.ytimg.com/vi/EFWP46evlvk/maxresdefault.jpg
[%] File Downloaded !

[%] Downloading Image #8 from https://huverfruit.es/wp-content/uploads/2015/04/tomate-maduro-fruteria-online-madrid.jpg
[%] File Downloaded !

[%] Downloading Image #9 from https://donadores.pt/354-large_default/tomate-maduro.jpg
[%] File Downloaded !

[%] Downloading Image #10 from https://comedelahuerta.com/wp-content/uploads/2019/07/tomate-maduro.jpg
[%] File Downloaded !

[%] Do

##2º passo - Pré-processamento

O pré-processamento de dados é uma etapa crucial no desenvolvimento de modelos de aprendizado de máquina, especialmente quando se trabalha com imagens.

 - Redimensionamento: Normalmente, é útil redimensionar todas as imagens para um tamanho padrão. Isso ajuda a garantir que todas as imagens tenham as mesmas dimensões, o que é importante para o treinamento eficaz do modelo. 


In [10]:
import os
from PIL import Image

def redimensionar_imagem(caminho_imagem, novo_tamanho):
    imagem = Image.open(caminho_imagem)
    imagem_redimensionada = imagem.resize(novo_tamanho)
    return imagem_redimensionada

def redimensionar_imagens(pasta_origem, novo_tamanho):
    # Percorre todas as imagens na pasta de origem
    for nome_imagem in os.listdir(pasta_origem):
        caminho_imagem_origem = os.path.join(pasta_origem, nome_imagem)
        
        try:
            # Tenta abrir a imagem
            imagem_redimensionada = redimensionar_imagem(caminho_imagem_origem, novo_tamanho)
        except Exception as e:
            print(f"Erro ao processar a imagem {caminho_imagem_origem}: {e}")
            continue
        
        # Converte a imagem para o modo RGB (se necessário)
        if imagem_redimensionada.mode == 'RGBA':
            imagem_redimensionada = imagem_redimensionada.convert('RGB')
        
        # Salva a imagem redimensionada na pasta de destino (sobrescreve a imagem original)
        caminho_imagem_destino = caminho_imagem_origem  # Salva na mesma pasta de origem
        imagem_redimensionada.save(caminho_imagem_destino)

# Definindo o novo tamanho das imagens (200x200 pixels)
novo_tamanho = (200, 200)

caminho_pasta_maduro = './dataset/tomate maduro'
redimensionar_imagens(caminho_pasta_maduro, novo_tamanho)

caminho_pasta_verde = './dataset/tomate verde'
redimensionar_imagens(caminho_pasta_verde, novo_tamanho)


##3º passo - Normalização
Esta etapa envolve escalar os valores dos pixels para que estejam dentro de uma faixa específica, como [0, 1] ou [-1, 1]. Isso ajuda a garantir que as características das imagens tenham uma escala consistente e facilita o treinamento do modelo. Você pode normalizar os pixels dividindo-os pelo valor máximo de intensidade (geralmente 255 para imagens RGB).

In [11]:
import os
import numpy as np
from PIL import Image

def carregar_imagens(caminho_pasta):
    imagens = []
    for nome_imagem in os.listdir(caminho_pasta):
        caminho_imagem = os.path.join(caminho_pasta, nome_imagem)
        imagem = Image.open(caminho_imagem)
        array_imagem = np.array(imagem)
        imagens.append(array_imagem)
    return np.array(imagens)

In [12]:
def calcular_estatisticas(imagens):
    #calcular a média e o desvio padrão
    media = np.mean(imagens, axis=(0,1,2))
    desvio_padrao = np.std(imagens, axis=(0,1,2))
    return media, desvio_padrao

In [13]:
def normalizar_imagens(imagens, media, desvio_padrao):
    #normalizar imagens
    imagens_normalizadas = (imagens - media) / desvio_padrao
    return imagens_normalizadas

In [14]:
caminho_pasta_maduro = './dataset/tomate maduro'
imagens_maduro = carregar_imagens(caminho_pasta_maduro)
media_maduro, desvio_padrao_maduro = calcular_estatisticas(imagens_maduro)
imagens_normalizadas_maduro = normalizar_imagens(imagens_maduro, media_maduro, desvio_padrao_maduro)


In [15]:
caminho_pasta_verde = './dataset/tomate verde'
imagens_verde = carregar_imagens(caminho_pasta_verde)
media_verde, desvio_padrao_verde = calcular_estatisticas(imagens_verde)
imagens_normalizadas_verde = normalizar_imagens(imagens_verde, media_verde, desvio_padrao_verde)


#Treinamento do modelo de aprendizado de máquina

##Divisão do Conjunto de Dados
Divisão do conjunto de dados em conjuntos de treinamento, validação e teste.

In [17]:
#criando array das imagens e rotulos para treinamento
import os
import numpy as np
from PIL import Image

def carregar_imagens_e_rotulos(pasta_tomate_maduro, pasta_tomate_verde):
    imagens = []
    rotulos = []

    #carregar imagens de tomates maduros
    for nome_imagem in os.listdir(pasta_tomate_maduro):
        caminho_imagem = os.path.join(pasta_tomate_maduro, nome_imagem)
        imagem = Image.open(caminho_imagem)
        imagens.append(np.array(imagem))#adicionar imagem como uma array numpy
        rotulos.append(1) #tomate maduro rotulado como 1

    #carregar imagens de tomates verdes
    for nome_imagem in os.listdir(pasta_tomate_verde):
        caminho_imagem = os.path.join(pasta_tomate_verde, nome_imagem)
        imagem = Image.open(caminho_imagem)
        imagens.append(np.array(imagem))
        rotulos.append(0) #tomate verde rotulado como 0

    return np.array(imagens), np.array(rotulos)

pasta_tomate_maduro = './dataset/tomate maduro'
pasta_tomate_verde = './dataset/tomate verde'

imagens, rotulos = carregar_imagens_e_rotulos(pasta_tomate_maduro, pasta_tomate_verde)

print("Número de imagens", len(imagens))
print("Número de rótulos: ",len(rotulos))

Número de imagens 112
Número de rótulos:  112


##Escolha do Modelo
Escolha do modelo de aprendizado de máquina para classificação de tomates.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Definição do modelo
modelo = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Saída binária (0 para tomate verde, 1 para tomate maduro)
])

# Compilação do modelo
modelo.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])

# Visualização da arquitetura do modelo
modelo.summary()


: 

    

    Treinamento do Modelo: Treine o modelo usando o conjunto de treinamento. Durante o treinamento, o modelo ajustará seus parâmetros para minimizar uma função de perda, usando um algoritmo de otimização como gradiente descendente. O desempenho do modelo será avaliado usando o conjunto de validação.

    Avaliação do Modelo: Avalie o desempenho do modelo usando o conjunto de teste. Calcule métricas de avaliação como precisão, recall, F1-score e matriz de confusão para avaliar o desempenho do modelo em classificar tomates verdes e maduros.

    Ajuste de Hiperparâmetros: Se necessário, ajuste os hiperparâmetros do modelo, como taxa de aprendizado, tamanho do lote, número de épocas, etc., com base no desempenho observado no conjunto de validação.

    Validação Cruzada: Considere a possibilidade de usar técnicas de validação cruzada para avaliar a robustez do modelo e reduzir a variância na estimativa de desempenho.

    Otimização do Modelo: Experimente técnicas de otimização de modelo, como ajuste fino de parâmetros, regularização, dropout, entre outras, para melhorar o desempenho do modelo.

    Implantação: Depois de treinar um modelo com desempenho satisfatório, você pode implantá-lo para uso prático, por exemplo, em um aplicativo ou sistema de automação.